## 訓練 CBoW 模型

此筆記本是 [AI for Beginners Curriculum](http://aka.ms/ai-beginners) 的一部分

在這個範例中，我們將學習如何訓練 CBoW 語言模型，以建立我們自己的 Word2Vec 嵌入空間。我們將使用 AG News 數據集作為文本來源。


In [ ]:
import torch
import torchtext
import os
import collections
import builtins
import random
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

首先讓我們載入資料集並定義分詞器和詞彙表。我們將 `vocab_size` 設定為 5000 以稍微限制計算量。


In [ ]:
def load_dataset(ngrams = 1, min_freq = 1, vocab_size = 5000 , lines_cnt = 500):
    tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
    print("Loading dataset...")
    test_dataset, train_dataset  = torchtext.datasets.AG_NEWS(root='./data')
    train_dataset = list(train_dataset)
    test_dataset = list(test_dataset)
    classes = ['World', 'Sports', 'Business', 'Sci/Tech']
    print('Building vocab...')
    counter = collections.Counter()
    for i, (_, line) in enumerate(train_dataset):
        counter.update(torchtext.data.utils.ngrams_iterator(tokenizer(line),ngrams=ngrams))
        if i == lines_cnt:
            break
    vocab = torchtext.vocab.Vocab(collections.Counter(dict(counter.most_common(vocab_size))), min_freq=min_freq)
    return train_dataset, test_dataset, classes, vocab, tokenizer

In [ ]:
train_dataset, test_dataset, _, vocab, tokenizer = load_dataset()

Loading dataset...
Building vocab...


In [ ]:
def encode(x, vocabulary, tokenizer = tokenizer):
    return [vocabulary[s] for s in tokenizer(x)]

## CBoW 模型

CBoW 學習根據 $2N$ 個相鄰的單詞來預測一個單詞。例如，當 $N=1$ 時，我們可以從句子 *I like to train networks* 中得到以下配對：(like, I)、(I, like)、(to, like)、(like, to)、(train, to)、(to, train)、(networks, train)、(train, networks)。在這裡，第一個單詞是作為輸入的相鄰單詞，第二個單詞是我們要預測的單詞。

為了構建一個用於預測下一個單詞的網絡，我們需要提供相鄰單詞作為輸入，並獲得單詞編號作為輸出。CBoW 網絡的架構如下：

* 輸入單詞會通過嵌入層。這個嵌入層就是我們的 Word2Vec 嵌入，因此我們會將其單獨定義為 `embedder` 變數。在這個例子中，我們將使用嵌入大小為 30，儘管你可能想嘗試更高的維度（真實的 Word2Vec 通常是 300 維）。
* 嵌入向量接著會傳遞到一個線性層，該層將預測輸出單詞。因此它有 `vocab_size` 個神經元。

對於輸出，如果我們使用 `CrossEntropyLoss` 作為損失函數，我們只需要提供單詞編號作為期望結果，而不需要使用 one-hot 編碼。


In [ ]:
vocab_size = len(vocab)

embedder = torch.nn.Embedding(num_embeddings = vocab_size, embedding_dim = 30)
model = torch.nn.Sequential(
    embedder,
    torch.nn.Linear(in_features = 30, out_features = vocab_size),
)

print(model)

Sequential(
  (0): Embedding(5002, 30)
  (1): Linear(in_features=30, out_features=5002, bias=True)
)


## 準備訓練數據

現在讓我們編寫主要函數，從文本中計算 CBoW 詞對。這個函數將允許我們指定窗口大小，並返回一組詞對——輸入詞和輸出詞。請注意，這個函數既可以用於詞，也可以用於向量/張量——這將使我們能夠在傳遞給 `to_cbow` 函數之前對文本進行編碼。


In [ ]:
def to_cbow(sent,window_size=2):
    res = []
    for i,x in enumerate(sent):
        for j in range(max(0,i-window_size),min(i+window_size+1,len(sent))):
            if i!=j:
                res.append([sent[j],x])
    return res

print(to_cbow(['I','like','to','train','networks']))
print(to_cbow(encode('I like to train networks', vocab)))

[['like', 'I'], ['to', 'I'], ['I', 'like'], ['to', 'like'], ['train', 'like'], ['I', 'to'], ['like', 'to'], ['train', 'to'], ['networks', 'to'], ['like', 'train'], ['to', 'train'], ['networks', 'train'], ['to', 'networks'], ['train', 'networks']]
[[232, 172], [5, 172], [172, 232], [5, 232], [0, 232], [172, 5], [232, 5], [0, 5], [1202, 5], [232, 0], [5, 0], [1202, 0], [5, 1202], [0, 1202]]


讓我們準備訓練數據集。我們將遍歷所有新聞，調用 `to_cbow` 來獲取詞對列表，並將這些詞對添加到 `X` 和 `Y` 中。為了節省時間，我們只考慮前 10k 條新聞項目——如果你有更多時間等待並希望獲得更好的嵌入，可以輕鬆移除這個限制 :)


In [ ]:
X = []
Y = []
for i, x in zip(range(10000), train_dataset):
    for w1, w2 in to_cbow(encode(x[1], vocab), window_size = 5):
        X.append(w1)
        Y.append(w2)

X = torch.tensor(X)
Y = torch.tensor(Y)

我們還將把該數據轉換為一個數據集，並創建數據加載器：


In [ ]:
class SimpleIterableDataset(torch.utils.data.IterableDataset):
    def __init__(self, X, Y):
        super(SimpleIterableDataset).__init__()
        self.data = []
        for i in range(len(X)):
            self.data.append( (Y[i], X[i]) )
        random.shuffle(self.data)

    def __iter__(self):
        return iter(self.data)

我們還將把該數據轉換為一個數據集，並創建數據加載器：


In [ ]:
ds = SimpleIterableDataset(X, Y)
dl = torch.utils.data.DataLoader(ds, batch_size = 256)

現在讓我們進行實際訓練。我們將使用 `SGD` 優化器，並設定相當高的學習率。你也可以嘗試使用其他優化器，例如 `Adam`。我們將先訓練 10 個世代——如果你希望更低的損失，可以重新執行此單元格。


In [ ]:
def train_epoch(net, dataloader, lr = 0.01, optimizer = None, loss_fn = torch.nn.CrossEntropyLoss(), epochs = None, report_freq = 1):
    optimizer = optimizer or torch.optim.Adam(net.parameters(), lr = lr)
    loss_fn = loss_fn.to(device)
    net.train()

    for i in range(epochs):
        total_loss, j = 0, 0, 
        for labels, features in dataloader:
            optimizer.zero_grad()
            features, labels = features.to(device), labels.to(device)
            out = net(features)
            loss = loss_fn(out, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss
            j += 1
        if i % report_freq == 0:
            print(f"Epoch: {i+1}: loss={total_loss.item()/j}")

    return total_loss.item()/j

In [ ]:
train_epoch(net = model, dataloader = dl, optimizer = torch.optim.SGD(model.parameters(), lr = 0.1), loss_fn = torch.nn.CrossEntropyLoss(), epochs = 10)

Epoch: 1: loss=5.664632366860172
Epoch: 2: loss=5.632101973960962
Epoch: 3: loss=5.610399051405015
Epoch: 4: loss=5.594621561080262
Epoch: 5: loss=5.582538017415446
Epoch: 6: loss=5.572900234519603
Epoch: 7: loss=5.564951676341915
Epoch: 8: loss=5.558288112064614
Epoch: 9: loss=5.552576955031129
Epoch: 10: loss=5.547634165194347


5.547634165194347

## 嘗試使用 Word2Vec

要使用 Word2Vec，我們來提取與詞彙表中所有單詞對應的向量：


In [ ]:
vectors = torch.stack([embedder(torch.tensor(vocab[s])) for s in vocab.itos], 0)

讓我們看看，例如，單詞**Paris**是如何被編碼成一個向量的：


In [ ]:
paris_vec = embedder(torch.tensor(vocab['paris']))
print(paris_vec)

tensor([-0.0915,  2.1224, -0.0281, -0.6819,  1.1219,  0.6458, -1.3704, -1.3314,
        -1.1437,  0.4496,  0.2301, -0.3515, -0.8485,  1.0481,  0.4386, -0.8949,
         0.5644,  1.0939, -2.5096,  3.2949, -0.2601, -0.8640,  0.1421, -0.0804,
        -0.5083, -1.0560,  0.9753, -0.5949, -1.6046,  0.5774],
       grad_fn=<EmbeddingBackward>)


使用 Word2Vec 來尋找同義詞是很有趣的。以下函數將返回與給定輸入最接近的 `n` 個詞。為了找到它們，我們計算 $|w_i - v|$ 的範數，其中 $v$ 是對應於我們輸入詞的向量，$w_i$ 是詞彙表中第 $i$ 個詞的編碼。我們接著對陣列進行排序，並使用 `argsort` 返回相應的索引，然後取列表的前 `n` 個元素，這些元素編碼了詞彙表中最接近詞的位置。


In [ ]:
def close_words(x, n = 5):
  vec = embedder(torch.tensor(vocab[x]))
  top5 = np.linalg.norm(vectors.detach().numpy() - vec.detach().numpy(), axis = 1).argsort()[:n]
  return [ vocab.itos[x] for x in top5 ]

close_words('microsoft')

['microsoft', 'quoted', 'lp', 'rate', 'top']

In [ ]:
close_words('basketball')

['basketball', 'lot', 'sinai', 'states', 'healthdaynews']

In [ ]:
close_words('funds')

['funds', 'travel', 'sydney', 'japan', 'business']

## 重點

使用像 CBoW 這樣的巧妙技術，我們可以訓練 Word2Vec 模型。你也可以嘗試訓練 skip-gram 模型，該模型是基於給定中心詞來預測鄰近詞，看看它的表現如何。



---

**免責聲明**：  
本文件已使用 AI 翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 進行翻譯。儘管我們努力確保翻譯的準確性，但請注意，自動翻譯可能包含錯誤或不準確之處。原始文件的母語版本應被視為權威來源。對於關鍵信息，建議使用專業人工翻譯。我們對因使用此翻譯而引起的任何誤解或誤釋不承擔責任。
